In [ ]:
#Gerekli Kütüphaneler Geliştirme Ortamına Dahil Ediliyor
import numpy as np # linear algebra
import pandas as pd # Veri işleme

# Visiualization tools
import matplotlib.pyplot as plt
import seaborn as sns

#Model Seçimi
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


#Makine Öğrenmesi Modelleri
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC




#Metrikler
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

from sklearn.externals import joblib

#Sistem Kütüphaneleri
import os
print(os.listdir("../input"))

In [ ]:
import warnings
#Sonuçların okunmasını zorlaştırdığı için uyarıları kapatıyoruz
warnings.filterwarnings("ignore")
print("Uyarılar Kapatıldı")

[İçindekiler Menüsüne Git](#0.)

<a class="anchor" id="2."></a>**Veri Seti Hakkında Bağlamsal Değerlendirme** 

Bu veri seti, Hindistan'ın Andhra Pradesh kentinin Kuzeydoğusundan toplanan 416 karaciğer hastası ve 167 karaciğer dışı hasta kaydını içermektedir. "Dataset" sütunu, grupları karaciğer hastası veya olmayan (hastalıksız) ayıran sınıflar içerir. Bu veri seti 441 erkek hasta kaydı ve 142 kadın hasta kaydını içermektedir.

Yaşları 89'u aşan herhangi bir hasta yaşı "90"  olarak belirlenmiştir.

Sütunlar:

* Hastanın yaşı
* Hastanın Cinsiyeti
* Toplam Bilirubin
* Doğrudan bilirubin
* Alkali Fosfotaz
* Alamin Aminotransferaz
* Aspartat Aminotransferaz
* Toplam Protiens
* Albümin
* Albumin ve Globulin Oranı
* Veri kümesi(Hedef değişken): alanı iki kümeye ayırmak için kullanılan alan (karaciğer hastalığı olan ya da hastalığı olmayan)



In [ ]:
#veri setini pandas DataFrame olarak yüklüyoruz
dataset=pd.read_csv('../input/indian_liver_patient.csv')

#veri setine ait ilk beş satır; 
dataset.head()

In [ ]:
# veri setindeki sayısal özelliklere ait özet istatistiksel bilgiler
# Gender özelliği sayısal olmayan değerler içerdiği için,istatistiksel verileri yoktur
dataset.describe().T

Yukarıdaki tabloda veri setinin istatiksel özeti vardır. Her bir özellik için adet, ortalam, standart sapma, minumum ve maksimum değerler içermektedir. Tablodaki değerler veri setinin özetini sunmakla beraber makine öğrenmesi modeli için bir anlam ifade etmemektedir. Sadece özelliklerin değer aralıklarının birbirinden çok farklı olduğu görünüyor olması, normalleştirme işleminin yapılması gerekitiğini gösteriyor. 

In [ ]:
dataset.info()

In [ ]:
#eksik veriler 'Albumin_and_Globulin_Ratio' sütununun ortalaması ile doldurulu
dataset['Albumin_and_Globulin_Ratio'].fillna(dataset['Albumin_and_Globulin_Ratio'].mean(), inplace=True)
dataset.info()

In [ ]:
#'Dataset' sütünun adı 'target' olarak değitiriliyor
dataset.rename(columns={'Dataset':'target'},inplace=True)
dataset.head()



# <a class="anchor" id="5.2."></a>**Cinsiyet Teşhisi ve  Yüzdeleri**  

countplot() kategorik değerlerin dağılımını özetlemede başarılı olsada, kategorik değerlerin yüzdelik dilimi hakkında açıklayıcı değildir. Pie chart yüzdelik dilimleri göstermede daha başarılıdır. Seaborn kütüphanesinde pie chart olmadığı için pandas kütüphanesinde yer alan pie çizim fonksiyonu kullanılacaktır.

In [ ]:
target_counts=dataset['target'].value_counts().values
gender_counts=dataset['Gender'].value_counts().values

fig1, axes=plt.subplots(nrows=1, ncols=2,figsize=(10,5))
fig1.suptitle("Teşhis ve Cinsiyet Yüzdeleri")

target_sizes=dataset.groupby('target').size()
axes[0].pie(
    x=target_counts,
    labels=['patient({})'.format(target_sizes[1]),'not patient({})'.format(target_sizes[2])],
    autopct='%1.1f%%'
)
axes[0].set_title("Hasta Teşhis Yüzdeleri")

gender_sizes=dataset.groupby('Gender').size()
axes[1].pie(
    x=gender_counts, 
    labels=['male({})'.format(gender_sizes['Male']), 'female({})'.format(gender_sizes['Female'])], 
    autopct="%1.1f%%"
)
axes[1].set_title("Hastaların Cinsiyet Yüzdeleri")

[İçindekiler Menüsüne Git](#0.)

# <a class="anchor" id="5.e5."></a>**Metin Değerlerinin Sayısal Değerlere Dönüştürülmesi**  

Bu noktadan sonra yapılacak görselleştirmeler için veri setindeki tüm özelliklerin sayısal değer içermesi gerekiyor. Veri setinde sadece 'Gender' özelliği sayısal değer içermemektedir. Bu özelliğin tutuğu değerlerin sayısal dönüştürülmesi gerekir. 'Gender' değişkeninde iki farklı kategorik değer vardır; Male, Female. Bu iki kategorik değeri nominal yapıdadır. Yani aralarında büyüklük küçüklük durumu yoktur. Bu nedenle OneHot Encoding yöntemiyle sayısala dönüştürülmesi gerekir.  

Pandas DataFrame nesnesinde yer alan get_dummies() fonksiyonu veri setinde string olan özelliklerin sayısal değere dönüşümünü OneHot Encoding yöntemiyle yapmaktadır. 

In [ ]:
dataset=pd.get_dummies(dataset)

In [ ]:
corr_matrix=dataset.corr()
fig, ax = plt.subplots(figsize=(12,12))
sns.heatmap(corr_matrix,annot=True,linewidths=.5, ax=ax)



# <a class="anchor" id="6."></a>**Makine Öğrenmesi Modelinin Kullanılması**  

Veri setinin makine öğrenmesi modeline verilmeden önce yapılması gereken işlemler:
1. Veri setinin data ve target olarak ayrıştırılması; X, y
1. Data kısmının normalleştirilmesi 
1. Data ve target ın eğitim ve test kümelerine ayrışltırılması


# <a class="anchor" id="6.1."></a>**Veri Setinin 'data' ve 'target' Olarak Ayrıştırılması**  

In [ ]:
#Veri seti data ve target olarak ayrıştırılır
X=dataset.drop('target', axis=1) #data
y=dataset['target'] # target



# <a class="anchor" id="6.3."></a>**Veri Setinin Eğitim ve Test Bölümlerine Ayrıştırılması**  

In [ ]:
#Eğitim ve test kümelerine ayrıştırılır
X_train, X_test, y_train, y_test=train_test_split(X,y, stratify=y, test_size=0.3,random_state=42)

[İçindekiler Menüsüne Git](#0.)

# <a class="anchor" id="6.4."></a>**Kullanılacak Makine Öğrenmesi Modelleri**  

In [ ]:
models=[]
models.append(("Logistic Regression",LogisticRegression()))
models.append(("Naive Bayes",GaussianNB()))
models.append(("KNN",KNeighborsClassifier(n_neighbors=5)))
models.append(("Decision Tree",DecisionTreeClassifier()))
models.append(("SVM",SVC()))


[İçindekiler Menüsüne Git](#0.)

# <a class="anchor" id="6.5."></a>**Kullanılan Makine Öğrenmesi Modelleri İçin Elde Edilen Sonuçlar**  

In [ ]:

for name, model in models:
    
    clf=model

    clf.fit(X_train, y_train)

    y_pred=clf.predict(X_test)
    print(10*"=","{} için Sonuçlar".format(name).upper(),10*"=")
    print("Accuracy:{:0.2f}".format(accuracy_score(y_test, y_pred)))
    print("Confusion Matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
    print("Classification Report:\n{}".format(classification_report(y_test,y_pred)))
    print(30*"=")